In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 1000)

# FashionID Dataset

## Attributes Analysis
Loads the data.csv file and prints the most common attributes (tags). Then creates columns for the selected attributes and converts them to dummy variables.

In [3]:
data_path = '../../../data/fashionid'
df = pd.read_csv(os.path.join(data_path, 'data.csv'), sep=';', encoding='utf-8')

In [4]:
df.describe()

,attributes,category,color,img_path,img_url,product_name
count,12295,12295,12295,12295,12295,12295
unique,11085,10,8,12295,11085,10713
top,"Damen Dirndlbluse von Krüger Dirndl,Materialmischung mit Baumwoll-Anteil,Kurzer Schnitt,Tiefer V-Ausschnitt mit Rüschenbesatz und Zierborte,Kurze, tief angesetzte Knopfleiste,1/2-Arm,Elastischer Saum,Rückenlänge bei Größe 36: 35 cm,Farbe Weiß,Artikelnummer: 9696045",kleider,blue,hosen/9793597.jpg,"https://img.fidcdn.net/r17/product/krueger-dirndl-dirndlbluse-mit-rueschen-und-zierborten-weiss_9696045,d2c0fb.jpg",Montego Kleid mit floralem Muster - Marineblau
freq,3,3194,3971,1,3,7


In [5]:
df['tags'] = df['attributes'].apply(lambda x: x.lower().split(','))

Get count for each of the tags, if it appears in the attributes list of any of the images.

In [6]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

Sort by the most used tags and print

In [7]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df[tag_df['ratio'] > 1]
tag_df = tag_df.sort_values('ratio', ascending=False)

In [8]:
tag_df['tag'].head(50)

21512                                      rundhalsausschnitt
18971                                        lockerer schnitt
23643                                     taillierter schnitt
16181                                           farbe schwarz
21347                                         reine baumwolle
492                                                      5 cm
23919         unser model ist 177 cm groß und trägt größe 36.
16193                                              farbe weiß
16131                                        farbe marineblau
18716                              leicht taillierter schnitt
18047                               knopf- und reißverschluss
21359                                           reine viskose
12130                                   baumwoll-elasthan-mix
16070                                        farbe dunkelblau
12186                     baumwollmischung mit stretch-anteil
23404                                          streifenmuster
23946   

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [9]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Schnitt

In [10]:
tag_df[tag_df['tag'].str.contains(' schnitt')]

,tag,count,ratio
18971,lockerer schnitt,2649,21.55
23643,taillierter schnitt,2314,18.82
18716,leicht taillierter schnitt,1147,9.33
12076,ausgestellter schnitt,750,6.10
16629,gerader schnitt,738,6.00
16023,eng anliegender schnitt,366,2.98
421,ärmelloser schnitt,213,1.73
18673,leicht ausgestellter schnitt,172,1.40
269,leicht taillierter schnitt,171,1.39
18465,körperbetonter schnitt,136,1.11


In [11]:
schnitt_list = tag_df[tag_df['tag'].str.contains(' schnitt')]['tag'].tolist()
df = create_tag_column(df, schnitt_list, 'passform')

In [12]:
df['passform'].unique()

array(['ärmelloser schnitt', nan, 'ausgestellter schnitt',
       'eng anliegender schnitt', 'taillierter schnitt',
       'leicht ausgestellter schnitt', 'leicht taillierter schnitt',
       'körperbetonter schnitt', 'lockerer schnitt', 'gerader schnitt'], dtype=object)

In [13]:
fit_loose = ['lockerer schnitt']
fit_normal = ['gerader schnitt', 'taillierter schnitt', 'ausgestellter schnitt']
fit_tight = ['eng anliegender schnitt', 'leicht ausgestellter schnitt', 'leicht taillierter schnitt', 
             'körperbetonter schnitt']
df['passform'] = ['loose' if x in fit_loose else 'normal' if x in fit_normal
                 else 'tight' if x in fit_tight else np.nan for x in df['passform']]

### Ausschnitt

In [14]:
tag_df[tag_df['tag'].str.contains('ausschnitt')]

,tag,count,ratio
21512,rundhalsausschnitt,4328,35.20
16550,geknöpfter schlüssellochausschnitt auf der rückseite,650,5.29
24058,v-ausschnitt,585,4.76
16697,gerippter rundhalsausschnitt,492,4.00
21536,rundhalsausschnitt mit eingesetztem v-ausschnitt,170,1.38
23737,tiefer rückenausschnitt,137,1.11
521,abgerundeter v-ausschnitt,125,1.02


In [15]:
ausschnitt_list = tag_df[tag_df['tag'].str.contains('ausschnitt')]['tag'].tolist()
df = create_tag_column(df, ausschnitt_list, 'ausschnitt')

In [16]:
df['ausschnitt'].unique()

array(['rundhalsausschnitt', nan, 'gerippter rundhalsausschnitt',
       'v-ausschnitt',
       'geknöpfter schlüssellochausschnitt auf der rückseite',
       'rundhalsausschnitt mit eingesetztem v-ausschnitt',
       'abgerundeter v-ausschnitt', 'tiefer rückenausschnitt'], dtype=object)

In [17]:
neckline_u = ['rundhalsausschnitt', 'gerippter rundhalsausschnitt', 
              'rundhalsausschnitt mit eingesetztem v-ausschnitt']
neckline_v = ['v-ausschnitt', 'abgerundeter v-ausschnitt']
neckline_back = ['geknöpfter schlüssellochausschnitt auf der rückseite', 'tiefer rückenausschnitt']
df['ausschnitt'] = ['u' if x in neckline_u else 'v' if x in neckline_v
                 else 'back' if x in neckline_back else np.nan for x in df['ausschnitt']]

### Ärmellänge

In [18]:
tag_df[tag_df['tag'].str.contains('ärmel')]

,tag,count,ratio
15498,dreiviertel-ärmel,600,4.88
18383,kurze ärmel,565,4.60
25003,ärmellos,513,4.17
18529,lange ärmel,443,3.60
16,angeschnittene ärmel,344,2.80
421,ärmelloser schnitt,213,1.73
25066,ärmellänge bei größe 36: 62 cm,163,1.33
21263,raglanärmel,152,1.24
25174,ärmellänge bei größe m: 63 cm,150,1.22
25176,ärmellänge bei größe m: 64 cm,140,1.14


In [19]:
aermel_list = tag_df[tag_df['tag'].str.contains('ärmel')]['tag'].tolist()
df = create_tag_column(df, aermel_list, 'ärmellänge')

In [20]:
df['ärmellänge'].unique()

array(['ärmelloser schnitt', nan, 'ärmellos', 'lange ärmel',
       'angeschnittene ärmel', 'kurze ärmel', 'dreiviertel-ärmel',
       'kappärmel', 'raglanärmel', '64\xa0cm', '60\xa0cm', '62\xa0cm',
       '63\xa0cm'], dtype=object)

Merging similiar attributes together

In [21]:
df['ärmellänge'] = ['ärmellos' if x == 'ärmelloser schnitt' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['ärmellos' if x == 'ärmelloser' else x for x in df['ärmellänge']]

Renaming attributes according to the aboutyou dataset

In [22]:
df['ärmellänge'] = ['langarm' if x == 'lange ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['viertelarm' if x == 'kurze ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['dreiviertelarm' if x == 'dreiviertel-ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['dreiviertelarm' if x == 'dreiviertel-ärmel mit ausgestellten abschlüssen' else x for x in df['ärmellänge']]

Deleting useless tags

In [23]:
df['ärmellänge'] = [x if x in ['ärmellos', 'langarm', 'viertelarm', 'dreiviertelarm'] else np.nan for x in df['ärmellänge']]

### Muster

In [24]:
tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]

,tag,count,ratio
23404,streifenmuster,834,6.78
16436,florales muster,588,4.78
662,allover-muster,584,4.75


In [25]:
muster_list = tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]['tag'].tolist()
df = create_tag_column(df, muster_list, 'muster')

In [26]:
df['muster'].unique()

array([nan, 'streifenmuster', 'florales muster', 'allover-muster'], dtype=object)

In [27]:
df['muster'] = ['gestreift' if x == 'streifenmuster' else x for x in df['muster']]
df['muster'] = ['geblümt/floral' if x == 'florales muster' else x for x in df['muster']]
df['muster'] = ['all-over-muster' if x == 'allover-muster' else x for x in df['muster']]
df['muster'] = ['spitze' if 'spitze' in str(x) else x for x in df['muster']]

In [28]:
df['muster'] = [x if x in ['gestreift', 'geblümt/floral', 'all-over-muster', 'spitze'] else np.nan for x in df['muster']]

## Create dummies
Turn the created attribute columns into dummy variables.

In [29]:
df = df.set_index('img_url')
df_dum = pd.get_dummies(df[['category', 'color', 'ärmellänge', 'muster', 'passform', 'ausschnitt']]).reset_index()

In [30]:
df_dum.head()

,img_url,category_blusen,category_hosen,category_jacken,category_jeans,category_jumpsuits,category_kleider,category_pullover-strick,category_roecke,category_shirts,...,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,passform_loose,passform_normal,passform_tight,ausschnitt_back,ausschnitt_u,ausschnitt_v
0,"https://img.fidcdn.net/r17/product/guess-kleid-porzia-mit-einsatz-aus-mesh-schwarz_9765638,3f8d67.jpg",0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,"https://img.fidcdn.net/r17/product/jakes-maxikleid-mit-streifenmuster-schwarz_9805106,bb47d7.jpg",0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,"https://img.fidcdn.net/r17/product/only-kleid-mit-rockteil-aus-floraler-spitze-schwarz_9815363,3aa47b.jpg",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"https://img.fidcdn.net/r17/product/only-one-shoulder-kleid-mit-volantbesatz-schwarz_9815368,5ac017.jpg",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"https://img.fidcdn.net/r17/product/windsor-etuikleid-mit-taillenguertel-schwarz_9875358,44d04f.jpg",0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


Join the relative image path based on the img_url and drop the img_url. This creates a dataframe that holds the relative image path and all its dummy variables.

In [31]:
df_dum = df_dum.groupby(['img_url']).max().reset_index()
df_paths = df.groupby(['img_url']).first()['img_path'].reset_index()
df_dum = df_paths.merge(df_dum, how='inner').drop(['img_url'], axis=1)

In [32]:
df_dum.head()

,img_path,category_blusen,category_hosen,category_jacken,category_jeans,category_jumpsuits,category_kleider,category_pullover-strick,category_roecke,category_shirts,...,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,passform_loose,passform_normal,passform_tight,ausschnitt_back,ausschnitt_u,ausschnitt_v
0,blusen/9767854.jpg,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,blusen/9767853.jpg,1,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
2,blusen/9767860.jpg,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,blusen/9767855.jpg,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,blusen/9767857.jpg,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [33]:
df_dum.to_csv(os.path.join(data_path, 'img_attr.csv'), sep='\t', encoding='utf-8', index=False)